<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/Gold__1_%26_2_%26_3_20251101_0800.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl
!pip install xlsxwriter
!pip install mlxtend
!pip caas_jupyter_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.6 MB/s eta 0:00:00
ERROR: unknown command "caas_jupyter_tools"


# Ron Rules-QUICK CODE CARD (v2025.10.27-Refined)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Deliver Python code (inline, executable, CUT-ready section)
# - XHTML 1.0 Transitional; old-school friendly; Times New Roman body.
# - Use CUT markers; five # spacer lines follow the STOP marker.

# Cell 1

In [2]:
# Cell 1 20250513-cell2 is good to use; adding more lineage functionality next
#!/usr/bin/env python
# RON RULES — QUICK CODE CARD (v2025.10.27-Refined)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Deliver Python code (inline, executable, CUT-ready section)
# - XHTML 1.0 Transitional; old-school friendly; Times New Roman body.
# - Use CUT markers; five # spacer lines follow the STOP marker.
"""
GEDCOM Composite Score Script using:
 - Chunk-based Parallel Processing for Speed (Stage 1: genealogical line creation)
 - A Trie-based approach, then final "Value" = 5 * (number of couples with node.count >=2) + (total couples)

For ancestral lines where none of the couples are repeated (a one-off line), the Value is still computed.
Now, instead of composite scoring, two new columns are added:
  - Value Range (the numeric bracket)
  - Value Label (a descriptive label)

Exports final CSV/HTML sorted by "Yates DNA Ancestral Line", including a 'haplogroup' column.
"""
import csv
import glob
import logging
import functools
import os
from datetime import datetime
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from IPython.display import display, Javascript

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

###############################################################################
# Global Variables
###############################################################################
anchor_gen1 = None
visited_pairs = set()
generation_table = []

###############################################################################
# Trie Data Structure
###############################################################################
class TrieNode:
    """A simple Trie node for storing a couple and counting how many lines pass here."""
    def __init__(self):
        self.count = 0
        self.children = {}

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert_line(self, couples_list):
        current = self.root
        for couple in couples_list:
            if couple not in current.children:
                current.children[couple] = TrieNode()
            current = current.children[couple]
            current.count += 1

    def get_couple_count(self, couples_list):
        counts = []
        current = self.root
        for couple in couples_list:
            if couple in current.children:
                current = current.children[couple]
                counts.append(current.count)
            else:
                counts.append(0)
                break
        return counts

###############################################################################
# Utility: chunk generator
###############################################################################
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

###############################################################################
# GedcomDataset
###############################################################################
class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1
        anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        elif '**' in npfx_value:
            cm_value = npfx_value.split('**')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return ''

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_part = npfx_value.split('&')[1]
            if '**' in sort_part:
                sort_value = sort_part.split('**')[0].strip()
            else:
                sort_value = sort_part.strip()
            return sort_value
        return ''

    def get_extractable_YDNA(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '**' in npfx_value:
            ydna_value = npfx_value.split('**')[1].strip()
            return ydna_value
        return ''

    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

###############################################################################
# Gedcom Class
###############################################################################
class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()

        current_dataset = None
        npfx_count = 0
        ydna_count = 0
        total_count = 0

        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_dataset.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)
                    if '**' in value:
                        ydna_count += 1

        autosomal_count = npfx_count - ydna_count
        print(f"GEDCOM contained {total_count} total records")
        print(f"Records tagged and filtered by NPFX: {npfx_count}")
        print(f"Records with YDNA information: {ydna_count}")
        print(f"Autosomal matches: {autosomal_count}")

        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)

        manual_filter_activated = True
        if manual_filter_activated:
            try:
                df = pd.read_excel('filtered_ids.xlsx')
            except FileNotFoundError:
                logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
            else:
                manual_filtered_ids = set(df['ID'])
                self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_filtered_ids]
                print(f"After manual filter, total records: {len(self.filter_pool)}")
                logger.info(f"After manual filter, total records: {len(self.filter_pool)}")

        return autosomal_count

###############################################################################
# quick_extract_name
###############################################################################
def quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "):
            idx = 0
        else:
            return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start)
    if end == -1:
        end = len(full_text)
    name_line = full_text[start:end].strip()
    if '/' not in name_line:
        return name_line[:10].replace(" ", "")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    return last_name[:10].replace(" ", "") + first_name[:10].replace(" ", "")

###############################################################################
# Parents & Ancestors
###############################################################################
def find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map:
        return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id:
        find_parents(father_id, generation+1, parents_map)
    if mother_id:
        find_parents(mother_id, generation+1, parents_map)

def find_distant_ancestors(individual_id, parents_map, path=None):
    if path is None:
        path = []
    path.append(individual_id)
    if individual_id not in parents_map:
        return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return [path]
    paths = []
    if father_id:
        paths.extend(find_distant_ancestors(father_id, parents_map, path[:]))
    if mother_id:
        paths.extend(find_distant_ancestors(mother_id, parents_map, path[:]))
    return paths if paths else [path]

###############################################################################
# filter_ancestral_line
###############################################################################
def filter_ancestral_line(winning_path_ids, generation_table_local, names_map):
    matching_table = []
    for generation, pair in generation_table_local:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    matching_table.sort(key=lambda x: x[0])
    lines = []
    for gen, pair in matching_table:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append(f"{name_pair[0]}&{name_pair[1]}")
    lines.reverse()
    return "~~~".join(lines)

###############################################################################
# process_record_wrapper (parallel) - STAGE 1
###############################################################################
def process_record_wrapper(individual_id, gedcom_instance, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []
    visited_pairs = set()

    find_parents(individual_id, 1, parents_map)
    distant_anc_paths = find_distant_ancestors(individual_id, parents_map)

    best_score = None
    best_path = None
    for path in distant_anc_paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score = score
            best_path = path

    if not best_path:
        best_path = []

    best_path_cleaned = [pid for pid in best_path if pid != individual_id]
    line_str = filter_ancestral_line(set(best_path_cleaned), generation_table, names_map)

    cm_value = ''
    sort_value = ''
    ydna_value = ''
    for ds in gedcom_instance.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value = ds.get_extractable_cm()
            sort_value = ds.get_extractable_sort()
            ydna_value = ds.get_extractable_YDNA()
            break

    short_name = names_map.get(individual_id, "UnknownName")
    # Return columns: ID#, Match to, Name, cM, Yates DNA Ancestral Line, haplogroup
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

###############################################################################
# main()
###############################################################################
def main():
    def select_gedcom():
        files = glob.glob("*.ged")
        if not files:
            print("No GEDCOM files found.")
            return None
        print("Automatically selecting the first GEDCOM file.")
        return files[0]

    gedcom_file_path = select_gedcom()
    if not gedcom_file_path:
        print("No GEDCOM file selected; exiting.")
        return

    ged = Gedcom(gedcom_file_path)
    autosomal_count = ged.parse_gedcom()
    filter_count = len(ged.filter_pool)

    with open("autosomal_count.txt", "w") as f:
        f.write(str(autosomal_count))

    print("Records tagged and filtered by NPFX:", filter_count)

    with open(gedcom_file_path, 'r', encoding='utf-8') as f:
        raw_data = f.read()

    blocks = raw_data.split('\n0 ')
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk:
            continue
        flend = blk.find('\n')
        if flend == -1:
            flend = len(blk)
        first_line = blk[:flend]
        if '@' in first_line:
            start = first_line.find('@') + 1
            end = first_line.find('@', start)
            rec_id = first_line[start:end].strip()
            all_records[rec_id] = blk

    parents_map = {}
    names_map = {}
    for rec_id, txt in all_records.items():
        nm = quick_extract_name("\n" + txt)
        names_map[rec_id] = nm

    families = {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)

    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print(f"Processing {len(individual_ids)} individuals with chunk-based parallel...")

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    logger.info("Starting chunk-based parallel processing with %d workers.", max_workers)

    with ProcessPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in chunks(individual_ids, chunk_size):
            func = functools.partial(process_record_wrapper, gedcom_instance=ged, parents_map=parents_map, names_map=names_map)
            results = list(executor.map(func, chunk))
            combined_rows.extend(results)
            pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    def remove_specific_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        if row["Yates DNA Ancestral Line"].startswith(prefix):
            row["Yates DNA Ancestral Line"] = row["Yates DNA Ancestral Line"][len(prefix):]
        return row

    df = df.apply(remove_specific_prefix, axis=1)

    logger.info("Building Trie from reversed lines...")
    trie = Trie()
    for _, row in df.iterrows():
        line_str = row["Yates DNA Ancestral Line"]
        if pd.notna(line_str) and line_str.strip():
            trie.insert_line([x.strip() for x in line_str.split("~~~") if x.strip()])

    values, prefix_counts = [], []
    logger.info("Computing 'Value' = 5*(#couples with node.count >=2) + (total couples) ...")
    for _, row in df.iterrows():
        line_str = row["Yates DNA Ancestral Line"]
        if pd.isna(line_str) or not line_str.strip():
            values.append(0)
            prefix_counts.append(0)
        else:
            couples_list = [x.strip() for x in line_str.split("~~~") if x.strip()]
            node_counts = trie.get_couple_count(couples_list)
            prefix_count = sum(1 for c in node_counts if c >= 2)
            values.append(5 * prefix_count + len(couples_list))
            prefix_counts.append(prefix_count)

    df["Value"], df["PrefixCount"] = values, prefix_counts

    def assign_value_range_label(val):
        try:
            v = float(val)
        except:
            return "", ""
        if v >= 60: return ">=60", "1-likely correct"
        if 47 <= v <= 59: return "59~47", "2-lines forming"
        if 34 <= v <= 46: return "46~34", "3-patterns emerging"
        if 21 <= v <= 33: return "33~21", "4-notable patterns"
        if 8 <= v <= 20: return "20~8", "5-patterns stable"
        if 1 <= v <= 7:  return f"{v:.0f}", "6-need research"
        return f"{v:.0f}", "0-uncategorized"

    ranges, labels = zip(*(assign_value_range_label(v) for v in df["Value"]))
    df["Value Range"], df["Value Label"] = ranges, labels

    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)
    df.drop("PrefixCount", axis=1, inplace=True)

    csv_name = "final_combined_df_with_value_labels.csv"
    df.to_csv(csv_name, index=False)
    logger.info("Exported final DataFrame to '%s'.", csv_name)

    html_name = "HTML_combined_df_with_value_labels.html"
    css_style = """
    <style>
    table { width: 100%; border-collapse: collapse; margin: 20px 0; }
    table, th, td { border: 1px solid #333; }
    th, td { padding: 8px 12px; text-align: center; }
    th { background-color: #f2f2f2; }
    /* Left-align the last column */
    td:nth-child(7) { text-align: left; }
    </style>
    """
    final_cols = ["ID#", "cM", "haplogroup", "Match to", "Value Range", "Value Label", "Yates DNA Ancestral Line"]
    html_content = css_style + df.to_html(index=False, columns=final_cols, escape=False)
    with open(html_name, "w", encoding="utf-8") as f:
        f.write(html_content)
    logger.info("Exported HTML to '%s'.", html_name)

if __name__ == '__main__':
    main()
    try:
        display(Javascript('alert("✅ GEDCOM processing (and HTML export) is complete!");'))
    except:
        pass

import os
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText

def send_email(subject, body, to_addr):
    smtp_server = 'smtp.gmail.com'
    port = 465
    sender = os.environ['GMAIL_USER']
    password = os.environ['GMAIL_APP_PASSWORD']
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = to_addr
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender, password)
        server.send_message(msg)







Automatically selecting the first GEDCOM file.
GEDCOM contained 62184 total records
Records tagged and filtered by NPFX: 1556
Records with YDNA information: 1
Autosomal matches: 1555
After manual filter, total records: 7
Records tagged and filtered by NPFX: 7
Processing 7 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 7/7 [00:03<00:00,  1.88it/s]


<IPython.core.display.Javascript object>

# Cell 2

In [3]:
# CELL 2
# ====== CUT START [1/1] INDEPENDENT CELL 2 — Build Register (Load -> Map -> HTML -> Save/Upload) ==
# RON RULES — QUICK CODE CARD (v2025.10.31-SelectByName)
# - Python code (inline, executable, CUT-ready section); ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Deliver full runnable code; no fabrication; no JSON/triple-quote artifacts.
# - XHTML 1.0 Transitional; old-school friendly; Times New Roman body.
# - Use CUT markers; five # spacer lines follow the STOP marker.

# ---------- 0) Imports & Secrets ----------
import os, re, io, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# Load secrets if running in Colab; otherwise respect existing env
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR'] = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR', '')

# ---------- 1) Config (filenames, layout, toggles) ----------
CSV_IN = "final_combined_df_with_value_labels.csv"
LOCAL_HTML = "ons_yates_dna_register.htm"
REMOTE_HTML = "ons_yates_dna_register.htm"
LOCAL_COUNT_FILE = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME = "autosomal_count.txt"
FTP_DIR = os.environ.get("FTP_DIR", "").strip()
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")
TNG_BASE = "https://yates.one-name.net/tng"
TNG_TREE = "tree1"
HOME_URL = "https://yates.one-name.net/ons_yates_dna_register.htm"
TABLE_WIDTH_PX = 3150
COL_A_PX = 1100
ARROW_ENTITY = "&rarr;"
REMOVE_PERIOD_AT_END = True
FIND_COL_PX = 118
UPLOAD_COLUMN_A = True

# Resolver on server
SERVER_PARTIALS_DIR = "partials"
SERVER_MAPPING_BASENAME = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"

# ---------- 2) FTP helpers ----------
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        try:
            ftps.cwd(FTP_DIR)
        except Exception:
            parts = [p for p in FTP_DIR.split("/") if p]
            for p in parts:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str, local_name: str) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"Pulled: {remote_name} -> {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        try:
            if os.path.exists(local_name): os.remove(local_name)
        except Exception: pass
        print(f"Skip pull (not found): {remote_name} ({e})")
        return False

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"Uploaded: {local_path} -> {remote_name}")

# ---------- 3) Resolver load (masked code -> unmasked display) ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1")
    last = None; df = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code","unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    with ftp_connect() as ftps:
        ok = ftp_download_if_exists(ftps, _remote_path(SERVER_MAPPING_REMOTE), SERVER_MAPPING_LOCAL_CACHE)
        try: ftps.quit()
        except Exception: pass
    if not ok:
        raise RuntimeError(
            "Resolver not found on server: /" + _remote_path(SERVER_MAPPING_REMOTE) +
            ". Upload match_to_unmasked.csv into /partials/ and re-run."
        )
    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"Loaded resolver: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))

MATCH_TO_UNMASKED = load_resolver_from_server()

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str): return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name utilities ----------
SEP_RE = re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")

def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s, str): s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text))
    t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}

def _smart_title(token: str) -> str:
    if not token: return token
    token = re.sub(r"(^|\b)([a-z])(['’])([a-z])", lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(), token.lower())
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])",  lambda m: "Mc"+m.group(1).upper(),  token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac"+m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name: return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

def surname_given_from_token(token):
    token = token.strip()
    if not token: return (token,)
    idx = None
    for i in range(1, len(token)):
        if token[i-1].islower() and token[i].isupper():
            idx = i; break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i; break
    if idx is None: return (token,)
    surname = token[:idx]; given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s): return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

def truncate_first(name: str, n: int = 4) -> str:
    name = name.strip()
    if not name: return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens: return ("","")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2: return ("","")
    def _norm(s):
        return smart_titlecase(s) if " " in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1: return ("parents" if g == 1 else "self")
    if g == 2: return "grandparents"
    greats = g - 2
    if greats == 1: return "great-grandparents"
    return f"{greats}x-great-grandparents"

def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try:
        cm_str = f"{int(round(float(cm_val)))}"
    except Exception:
        cm_str = (str(cm_val).strip() or "0")
    degree_label = degree_label_from_generations(gens)
    parts = [
        f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
        f"{degree_label} (back {gens} Gens)",
        "are",
        f"{husband} & {wife}."
    ]
    s = " ".join(parts)
    if REMOVE_PERIOD_AT_END: s = re.sub(r'\.\s*$', '', s)
    return s

# ---------- 5) Read main CSV; detect columns ----------
def find_col(df, patterns, prefer_exact=None):
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns: return name
            if name and name.lower() in lowmap: return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c): return c
    return None

_encs = ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"Loaded main CSV: {len(df)} rows, {len(df.columns)} cols")

id_col = find_col(df, [r'^(id#|personid)$'], ["ID#","ID","PersonID","personID"])
match_to_col = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col = find_col(df, [r'^name$'], ["Name"])
cm_col = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],
                    ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col: raise ValueError("CSV missing an ID#/PersonID column.")
if not match_to_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col: raise ValueError("CSV missing 'Name' column.")
if not cm_col: raise ValueError("CSV missing 'cM' column.")
if not path_col: raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform rows -> display_df ----------
ID_PAT = re.compile(r"\bI\d+\b", re.I)
def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def _truncate_alpha(s: str, n: int) -> str:
    return re.sub(r"[^A-Za-z]", "", s)[:n]

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw: return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1: return nm
        given = parts[0]; surname = parts[-1]
        return f"{_truncate_alpha(given, 7)} {surname}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1: return nm
        return f"{_truncate_alpha(ps[0], 7)} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates: return surname
    given = smart_titlecase(given_candidates[0])
    return f"{_truncate_alpha(given, 7)} {surname}".strip()

headers, lineages, findcol = [], [], []
subjects, first_ancestors = [], []
REMOTE_NAME_ABS = "/" + REMOTE_HTML

for _, row in df.iterrows():
    subject_raw = row.get(match_to_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name

    if pid:
        matchee_name_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank">{matchee_name}</a>'
        )
    else:
        matchee_name_html = matchee_name

    cm_val = row.get(cm_col, "0")

    tokens = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]

    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband","")).strip()
        wife_raw = str(row.get("common_wife","")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_name_html,
        gens_total,
        truncate_first(husband_raw, 7) if husband_raw else "",
        truncate_first(wife_raw, 7) if wife_raw else ""
    )

    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    q = _u.quote(subject_name)
    quick = (
        f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
        f'title="Open a filtered view for {subject_name}">Find</a>'
    )

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)
    subjects.append(subject_name)
    first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE = "Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"] = headers
df[LINEAGE_HEADER_SAFE] = lineages
df["Find"] = findcol
df["Subject"] = subjects
df["First Ancestor"] = [ _clean_piece(x) for x in first_ancestors ]
display_df = df[["Find","Match Summary", LINEAGE_HEADER_SAFE]]

# Column A CSV (optional upload)
MATCH_COUSINS_CSV = "the_match_cousins.csv"
display_df[["Match Summary"]].to_csv(MATCH_COUSINS_CSV, index=False, encoding="iso-8859-15")
print("Wrote:", os.path.abspath(MATCH_COUSINS_CSV))

# ---------- 7) HTML (Times, sticky headers, sortable, back-to-top) ----------
html_table = display_df.to_html(index=False, escape=False, classes="sortable")
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable">',
    '<table border="1" class="dataframe sortable" id="refactor-table">', 1
)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)

# Header: rename "Find" -> "Select:" (no header checkbox)
html_table = html_table.replace("<th>Find</th>", '<th>Select:</th>', 1)

# Sort-hint headers
html_table = html_table.replace("<th>Match Summary</th>", "><th>Match Summary&ndash;click to sort</th>", 1).replace("><th>Match Summary&ndash;click to sort</th>", "<th>Match Summary&ndash;click to sort</th>", 1)
html_table = html_table.replace(f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>", "<th>Lineage (Starting with oldest ancestor&ndash;click to sort)</th>", 1)

colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html, 1
)

TABLE_CSS = (
    "<style type=\"text/css\">\n"
    "  html { scroll-behavior: smooth; }\n"
    "  body { font-family: 'Times New Roman', Georgia, serif; font-size:100%; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }\n"
    f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }}\n"
    "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
    "  h1 { margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }\n"
    "  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }\n"
    "  .sortbar { margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px; display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; }\n"
    "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }\n"
    "  .btn:hover { background:#4668aa; }\n"
    "  input.btn.search { background:#fff; color:#111; border-color:#bbb; }\n"
    "  .find-cell { white-space:nowrap; }\n"
    "  .selbox { margin-right:6px; vertical-align:middle; }\n"
    "  .table-scroll { max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }\n"
    f"  table.sortable {{ border-collapse:collapse; width:{TABLE_WIDTH_PX}px; table-layout:fixed; }}\n"
    "  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }\n"
    "  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }\n"
    "  #first-row td { border-top:2px solid #999; }\n"
    "  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff; cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }\n"
    "  .back-to-top:hover { background:#4668aa; }\n"
    "  #dynamicContent { margin:10px 0 14px 0; }\n"
    "  @media screen and (max-width: 820px) { .wrap { padding:12px; } h1 { font-size:22px; } }\n"
    "</style>\n"
)

# Toolbar (no Email). Add Show Selected / Show All for multi-name aggregation.
DYNAMIC_BLOCK = (
    "<div class=\"sortbar\">\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_cousin_surname_study.htm\" target=\"_blank\">Study Details</a>\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/gengen/dna_theory_of_the_case.htm\" target=\"_blank\">Theory in Action</a>\n"
    "  <a class=\"btn\" href=\"gengen/images/cousin-calculator.jpg\" target=\"_blank\">Cousin Connection</a>\n"
    "  <a class=\"btn\" href=\"gengen/images/Shared_cM_Project_v4.jpg\" target=\"_blank\">Cousin by DNA</a>\n"
    "  <a class=\"btn\" href=\"partials/match_count.htm\" target=\"_blank\" rel=\"noopener\">Match Count</a>\n"
    "  <a class=\"btn\" href=\"partials/lineage_count.htm\" target=\"_blank\" rel=\"noopener\">Lineage Count</a>\n"
    "  <a class=\"btn\" href=\"https://yates.one-name.net/yates_ancestor_register.htm\">Ancestor Register</a>\n"
    "  <span class=\"btn\" id=\"show-selected\" title=\"Show all rows for the checked name(s)\">Show Selected</span>\n"
    "  <span class=\"btn\" id=\"show-all\" title=\"Show all rows\">Show All</span>\n"
    "  <span class=\"btn\" id=\"print-cousin-list\" style=\"cursor:pointer;\" title=\"Open a printable list of the *currently visible* rows\">Cousin List (Printable)</span>\n"
    "  <span class=\"btn\" id=\"clear-selected\">Clear</span>\n"
    "  <input type=\"text\" id=\"search-box\" class=\"btn search\" size=\"24\" value=\"\" placeholder=\"Search&hellip;\" />\n"
    "</div>\n"
    "<div id=\"dynamicContent\"></div>\n"
)

JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")
UPDATED_BLOCK = (
    "<div class=\"updated\">"
    f"<a href=\"{HOME_URL}\" target=\"_blank\" rel=\"noopener\">Home</a>"
    " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
    " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\" class=\"js-count\"></span>"
    " &nbsp;|&nbsp; Showing: <span id=\"showing-count\"></span>"
    "</div>"
)

# ---------- 8) Build final HTML via string.Template ----------
page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$TABLE_CSS
</head>
<body id="top">
<div class="wrap">
  <h1>ONS Yates Study Autosomal DNA Register</h1>
  $UPDATED_BLOCK
  $DYNAMIC_BLOCK
  <div class="table-scroll">
    $HTML_TABLE
  </div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]);
    var asc=(dir==='asc');
    rows.sort(function(a,b){
      var A=textOf(a.cells[colIndex]), B=textOf(b.cells[colIndex]);
      if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;
    });
    var frag=document.createDocumentFragment();
    for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]);
    tb.appendChild(frag);
    updateShowing();
  }

  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx];
      var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[↑↓]/,''); }
        th.innerHTML += (dir==='asc' ? ' &uarr;' : ' &darr;');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }

  function stampLastUpdated(){
    var el=document.getElementById('last-updated'); if(!el) return;
    var d=new Date(document.lastModified||new Date());
    function z(n){return(n<10?'0':'')+n;}
    el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());
  }

  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='$JS_COUNT_URL';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){ if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; } } };
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }

  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0;
    for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; }
    return n;
  }
  function updateShowing(){
    var el=document.getElementById('showing-count'); if(!el) return;
    el.textContent = formatWithCommas(visibleRowCount());
  }

  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q=norm(q);
      for(var i=0;i<cached.length;i++){ var hit = !q || cached[i].txt.indexOf(q)>-1; cached[i].el.style.display = hit? '' : 'none'; }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    box.addEventListener('search', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }

  // Collect all checkboxes
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var cb=tb.rows[i].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }

  // When you check any row for a given name, sync all rows with that same name.
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      if(!(e.target && e.target.classList && e.target.classList.contains('selbox'))) return;
      var nm = e.target.getAttribute('data-name') || '';
      var checked = !!e.target.checked;
      var cbs = allRowCheckboxes();
      for(var i=0;i<cbs.length;i++){
        if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; }
      }
    }, false);
  }

  // Show all matches for the selected name(s)
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs = allRowCheckboxes();
      var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox');
        var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }

  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }

  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }

  function bindBackToTop(){
    var btn=document.getElementById('back-to-top'); if(!btn) return;
    window.addEventListener('scroll', function(){ btn.style.display = (window.scrollY>200) ? 'block' : 'none'; }, {passive:true});
    btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
  }

  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\s*/, '');
      }
    }
  }

  // Print uses the currently visible rows (works with Show Selected)
  function bindPrintCousinList(){
    var btn=document.getElementById('print-cousin-list'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!tbl) return;
      var headerHtml = '';
      try{ headerHtml = tbl.tHead.innerHTML; }catch(e){}

      var tb=tbl.tBodies[0];
      var visibleRowsHtml = '';
      var visibleCount = 0;
      if(tb){
        for (var i = 0; i < tb.rows.length; i++) {
          if (tb.rows[i].style.display !== 'none') {
            visibleRowsHtml += tb.rows[i].outerHTML;
            visibleCount++;
          }
        }
      }

      var css = '<style type="text/css">' +
        "body { font-family: 'Times New Roman', Georgia, serif; font-size:12px; margin: 20px; }" +
        "h1 { font-size:20px; text-align:center; }" +
        "table { border-collapse:collapse; width:100%; table-layout:fixed; }" +
        "th, td { border:1px solid #999; padding: 5px 7px; vertical-align:top; text-align:left; word-wrap:break-word; }" +
        "th { background:#f0f0f0; }" +
        "a { color:#000; text-decoration:none; }" +
        "th:first-child, td:first-child { display:none; }" +
        "th:nth-child(2), td:nth-child(2) { width: 40% !important; }" +
        "th:nth-child(3), td:nth-child(3) { width: 60% !important; }" +
        '</style>';

      var tableHtml = '<table border="1">' + '<thead>' + headerHtml + '</thead><tbody>' + visibleRowsHtml + '</tbody></table>';

      var docHtml = '<html><head><title>Cousin List (Filtered)</title>' + css + '</head><body onload="window.print(); window.close();">' +
                    '<h1>Cousin List</h1>' +
                    '<p>Showing ' + visibleCount + ' filtered records.</p>' +
                    tableHtml +
                    '</body></html>';

      var win = window.open('', 'CousinPrint');
      win.document.open();
      win.document.write(docHtml);
      win.document.close();
      win.focus();
    }, false);
  }

  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){} }

  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    stampLastUpdated();
    loadAutoCount();
    bindHeaderSort();
    bindSearch();
    bindGroupSync();
    bindShowSelected();
    bindShowAll();
    bindClear();
    bindBackToTop();
    bindPrintCousinList();
    initShowingStatic();
  });
})();
 //]]>
</script>
</body>
</html>
""")

final_html = page_tpl.safe_substitute(
    TABLE_CSS=TABLE_CSS,
    UPDATED_BLOCK=UPDATED_BLOCK,
    DYNAMIC_BLOCK=DYNAMIC_BLOCK,
    HTML_TABLE=html_table,
    JS_COUNT_URL=JS_COUNT_URL
)

# ---------- 8.1) Partials builder (match_count.htm + lineage_count.htm) ----------
def _norm_code_for_count(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = t.replace("\u00a0"," ").strip()
    t = re.sub(r"\s{2,}"," ",t)
    return t.lower()

def _partial_css_wrapper():
    return (
        "<style type=\"text/css\">\n"
        "  html { scroll-behavior: smooth; }\n"
        "  body { font-family:'Times New Roman', Georgia, serif; background:#ffffff; color:#222; margin:0; padding:0; }\n"
        f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:32px; }}\n"
        "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
        "  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }\n"
        "  .updated { font-size:12px; color:#555; text-align:center; margin:4px 0 12px 0; }\n"
        "  .table-scroll { max-height:75vh; overflow:auto; border:1px solid #ddd; }\n"
        "  table.sortable { border-collapse:collapse; width:100%; min-width:720px; }\n"
        "  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; }\n"
        "  table.sortable th { background:#e3eaf8; position:sticky; top:0; text-align:left; z-index:2; }\n"
        "</style>\n"
    )

def _partial_header_block(showing_placeholder_js_ready):
    return (
        "<div class=\"updated\">"
        f"<a href=\"{HOME_URL}\" target=\"_blank\" rel=\"noopener\">Home</a>"
        " &nbsp;|&nbsp; Last updated: <span id=\"last-updated\"></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\"></span>"
        f" &nbsp;|&nbsp; Showing: <span id=\"showing-count\">{showing_placeholder_js_ready}</span>"
        "</div>\n"
    )

def _partial_js_block():
    return (
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){\n"
        " function z(n){return (n<10?'0':'')+n;}\n"
        " function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}\n"
        f" var URL='{JS_COUNT_URL}';\n"
        " function fmt(n){try{var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US');}catch(e){return String(n||'')}}\n"
        " function load(){var el=document.getElementById('auto-count'); if(!el) return; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=fmt(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}\n"
        " function setShowing(){var el=document.getElementById('refactor-table'); var n=0; try{var rows=el.tBodies[0].rows; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; }}catch(e){}; var sc=document.getElementById('showing-count'); if(sc) sc.textContent = fmt(n); }\n"
        " document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); setShowing(); }, false);\n"
        "})();\n"
        "//]]>\n</script>\n"
    )

def _partial_shell(title, table_html, showing_for_initial):
    css = _partial_css_wrapper()
    header = _partial_header_block(showing_for_initial)
    js = _partial_js_block()
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
        f"{css}</head>\n<body>\n"
        "<div class=\"wrap\">\n"
        f"  <h1>{_html.escape(title)}</h1>\n"
        f"  {header}"
        "  <div class=\"table-scroll\">\n"
        f"    {table_html}\n"
        "  </div>\n"
        "</div>\n"
        f"{js}</body>\n</html>"
    )

def build_and_write_partials(main_df: pd.DataFrame):
    os.makedirs("partials", exist_ok=True)

    # MATCH COUNT
    codes_raw = main_df[match_to_col].astype(str).map(lambda x: x.strip())
    keys_norm = codes_raw.map(_norm_code_for_count)

    counts_series = keys_norm.value_counts(dropna=False)
    counts = counts_series.reset_index()
    if counts.shape[1] >= 2:
        counts.columns = ["norm_key", "Count"]
    else:
        counts["norm_key"] = counts.index.astype(str)
        counts["Count"] = counts_series.values
        counts = counts[["norm_key","Count"]]

    first_display = {}
    for code_disp, k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k) != "":
            first_display[k] = code_disp

    counts["Code"] = counts["norm_key"].map(lambda k: first_display.get(k, ""))
    counts["Unmasked"] = counts["norm_key"].map(lambda k: MATCH_TO_UNMASKED.get(k, ""))

    counts = counts.sort_values(by=["Code","Count"], ascending=[True, False], kind="mergesort").reset_index(drop=True)
    match_df = counts[["Code","Unmasked","Count"]]

    match_tbl = match_df.to_html(index=False, classes="sortable", border=1, table_id="refactor-table")
    match_html = _partial_shell("Match Count", match_tbl, "0")
    mc_local = os.path.join("partials", "match_count.htm")
    with open(mc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(match_html)
    print("[OK] Wrote partial:", mc_local)

    # LINEAGE COUNT
    first_series = main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x: x.strip())
    vc = first_series[first_series != ""].value_counts(dropna=False)
    lin_df = vc.reset_index()
    if lin_df.shape[1] >= 2:
        lin_df.columns = ["First Ancestor","Count"]
    else:
        lin_df["First Ancestor"] = lin_df.index.astype(str)
        lin_df["Count"] = vc.values
        lin_df = lin_df[["First Ancestor","Count"]]
    lin_df = lin_df.sort_values(["Count","First Ancestor"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

    lin_tbl = lin_df.to_html(index=False, classes="sortable", border=1, table_id="refactor-table")
    lin_html = _partial_shell("Lineage Count", lin_tbl, "0")
    lc_local = os.path.join("partials", "lineage_count.htm")
    with open(lc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(lin_html)
    print("[OK] Wrote partial:", lc_local)

    # COUSIN LIST (PRINTABLE - FULL)
    cousin_df = main_df[["Match Summary"]].copy()
    cousin_df = cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)

    cousin_tbl = cousin_df.to_html(index=False, escape=False, classes="sortable", border=1, table_id="refactor-table")

    cousin_html = _partial_shell("Cousin List (Printable)", cousin_tbl, "0")
    cl_local = os.path.join("partials", "cousin_list_print.htm")
    with open(cl_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# Build partials now (based on full df with helper columns)
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

# ---------- 9) Save locally ----------
with open(LOCAL_HTML, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html)
print("Saved:", os.path.abspath(LOCAL_HTML))

# ---------- 10) Uploads (main page, Column A CSV, autosomal count if present, partials) ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("Missing FTP creds; skipping uploads.")
        return
    try:
        ftps = ftp_connect()
        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, REMOTE_HTML)
        except Exception as e:
            print("Upload main HTML failed:", e)
        if UPLOAD_COLUMN_A and os.path.exists(MATCH_COUSINS_CSV):
            try:
                ftp_upload_overwrite(ftps, MATCH_COUSINS_CSV, MATCH_COUSINS_CSV)
            except Exception as e:
                print("Upload Column A CSV failed:", e)
        if os.path.exists(LOCAL_COUNT_FILE):
            try:
                ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, REMOTE_COUNT_NAME)
            except Exception as e:
                print("Upload autosomal count failed:", e)

        # Upload partials to /partials/
        try:
            ftp_upload_overwrite(ftps, PARTIAL_MATCH_LOCAL, posixpath.join("partials","match_count.htm"))
        except Exception as e:
            print("Upload match_count.htm failed:", e)
        try:
            ftp_upload_overwrite(ftps, PARTIAL_LINEAGE_LOCAL, posixpath.join("partials","lineage_count.htm"))
        except Exception as e:
            print("Upload lineage_count.htm failed:", e)

        # COUSIN LIST PARTIAL
        try:
            ftp_upload_overwrite(ftps, PARTIAL_COUSIN_LOCAL, posixpath.join("partials","cousin_list_print.htm"))
        except Exception as e:
            print("Upload cousin_list_print.htm failed:", e)

        try: ftps.quit()
        except Exception: pass
        print("Uploads complete.")
    except Exception as e:
        print("FTP session failed:", e)
        traceback.print_exc()

save_and_upload_all()
#
#
#
#
#
# ====== CUT STOP [1/1] INDEPENDENT CELL 2 ==



<>:658: SyntaxWarning: invalid escape sequence '\s'
<>:658: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-417621202.py:658: SyntaxWarning: invalid escape sequence '\s'
  cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\s*/, '');


Pulled: partials/match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
Loaded resolver: 79 codes
Loaded main CSV: 7 rows, 9 cols
Wrote: /content/the_match_cousins.csv
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
Saved: /content/ons_yates_dna_register.htm
Uploaded: ons_yates_dna_register.htm -> ons_yates_dna_register.htm
Uploaded: the_match_cousins.csv -> the_match_cousins.csv
Uploaded: /content/autosomal_count.txt -> autosomal_count.txt
Uploaded: partials/match_count.htm -> partials/match_count.htm
Uploaded: partials/lineage_count.htm -> partials/lineage_count.htm
Uploaded: partials/cousin_list_print.htm -> partials/cousin_list_print.htm
Uploads complete.


# Cell 3

In [4]:
# Gold 3 Ancestor Register (Dynamic header + live counts)
# ====== CUT START [1/6] RULES + IMPORTS + SECRETS ==============================================
# RON RULES — QUICK CODE CARD (v2025.10.27-Refined)
# 1) EXECUTION: Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# 2) PUNCTUATION IN STRINGS: Use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# 3) CONTENT: Deliver full runnable code (no snippets). No fabrication or inference.
# 4) Python code (inline, executable, CUT-ready section)
# 5) HTML: XHTML 1.0 Transitional style acceptable; avoid HTML5-only tags if not needed.
# 6) INTEGRITY: Work in CUT-ready sections only; exactly five # lines after each section.

import os, re, socket, traceback
from ftplib import FTP_TLS
from datetime import datetime
from zoneinfo import ZoneInfo
import html as _html
import pandas as pd
# ====== CUT STOP  [1/6] RULES + IMPORTS + SECRETS ===============================================
#####
#####
#####
#####
#####


# ====== CUT START [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ===================================
# --- Securely load secrets (Colab or env) ---
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_DIR  = os.environ.get('FTP_DIR', '').strip().strip('/')
COUNT_PUBLIC_URL = ("/%s/%s" % (FTP_DIR, "autosomal_count.txt")) if FTP_DIR else "/autosomal_count.txt"

# Inputs
INPUT_CSV   = "final_combined_df_with_value_labels.csv"
OUTPUT_NAME = "yates_ancestor_register.htm"
HOME_URL    = "https://yates.one-name.net/ons_yates_dna_register.htm"

# Load CSV
try:
    df = pd.read_csv(INPUT_CSV)
    print("[OK] Loaded CSV:", INPUT_CSV, f"rows={len(df)}, cols={len(df.columns)}")
except FileNotFoundError:
    raise SystemExit("[ERROR] Missing file: final_combined_df_with_value_labels.csv")
except Exception as e:
    print("[ERROR] Failed to load CSV:", e)
    raise SystemExit("CSV loading failed.")

# Normalize haplogroup column presence
if 'haplogroup' not in df.columns:
    df['haplogroup'] = ''
else:
    df['haplogroup'] = df['haplogroup'].fillna('')

# Read autosomal count locally if present (fallback display only)
autosomal_count = None
try:
    with open("autosomal_count.txt", "r") as f:
        autosomal_count = int(f.read().strip())
except Exception:
    pass

# Optional delta vs previous run
prev_count, additional_str = None, ""
if os.path.exists("autosomal_count_prev.txt"):
    try:
        with open("autosomal_count_prev.txt", "r") as f:
            prev_count = int(f.read().strip())
        if autosomal_count is not None and prev_count is not None:
            diff = autosomal_count - prev_count
            if diff != 0:
                additional_str = " (+%d since last run)" % diff
    except Exception:
        pass

# Human-readable fallback timestamp (not shown; JS will stamp dynamically)
now = datetime.now(ZoneInfo("America/New_York"))
updated_fallback = now.strftime("%Y-%m-%d %H:%M")
# ====== CUT STOP  [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ====================================
#####
#####
#####
#####
#####


# ====== CUT START [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
# Column letters in MAIN df:
#   A = ID#
#   B = match to (masked)
#   C = Unmasked Name (output)

A_IDX = 0
B_IDX = 1
C_IDX = 2

def _norm_code(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = t.replace("\u00a0", " ").strip()
    t = re.sub(r"\s{2,}", " ", t)
    return t.lower()

REMOTE_PATH = "partials/match_to_unmasked.csv"
LOCAL_PATH  = "match_to_unmasked.csv"

# Pull resolver if not present
if not os.path.exists(LOCAL_PATH):
    print("Pulling resolver CSV from server...")
    with FTP_TLS(timeout=30) as ftps:
        ftps.connect(os.environ.get("FTP_HOST",""), int(os.environ.get("FTP_PORT","21")))
        ftps.login(os.environ.get("FTP_USER",""), os.environ.get("FTP_PASS",""))
        try: ftps.prot_p()
        except Exception: pass
        try: ftps.set_pasv(True)
        except Exception: pass
        if FTP_DIR:
            for p in FTP_DIR.split("/"):
                if not p: continue
                try: ftps.cwd(p)
                except Exception:
                    try: ftps.mkd(p)
                    except Exception: pass
                    ftps.cwd(p)
        try: ftps.cwd("partials")
        except Exception: pass
        with open(LOCAL_PATH, "wb") as f:
            ftps.retrbinary("RETR match_to_unmasked.csv", f.write)
    print("Resolver saved:", os.path.abspath(LOCAL_PATH))
else:
    print("Using cached resolver:", os.path.abspath(LOCAL_PATH))

def _load_resolver(path):
    last_err = None
    for enc in ("utf-8-sig","iso-8859-15","utf-8","cp1252","latin1"):
        try:
            m = pd.read_csv(path, dtype=str, keep_default_na=False, encoding=enc)
            break
        except Exception as e:
            last_err = e
            m = None
    if m is None:
        raise RuntimeError("Unable to read resolver CSV: %s (%r)" % (path, last_err))
    cols = {c.lower(): c for c in m.columns}
    if "code" not in cols or "unmasked" not in cols:
        raise ValueError("Resolver CSV must have columns: code, unmasked")
    m = m[[cols["code"], cols["unmasked"]]].copy()
    m["__key__"] = m[cols["code"]].map(_norm_code)
    m["__val__"] = m[cols["unmasked"]].astype(str)
    m = m.drop_duplicates(subset="__key__", keep="first")
    return dict(zip(m["__key__"], m["__val__"]))

resolver_map = _load_resolver(LOCAL_PATH)

if df.shape[1] < 3:
    raise ValueError("Main df must have at least 3 columns: A(ID#), B(match to), C(unmasked).")

masked_raw = df.iloc[:, B_IDX].astype(str)
masked_key = masked_raw.map(_norm_code)
resolved   = masked_key.map(resolver_map)

df.iloc[:, C_IDX] = resolved.fillna("")

mapped = int(resolved.notna().sum())
total  = len(df)
print("[OK] Column B -> C mapping:", mapped, "/", total, "unmatched:", total - mapped)
# ====== CUT STOP  [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
#####
#####
#####
#####
#####


# ====== CUT START [4/6] XHTML TEMPLATE + TABLE (Dynamic header + counts + search) ===============
_BTN_BG   = "#5b79b8"
_BTN_BG_H = "#4668aa"
_TH_BG    = "#e3eaf8"
_LINK     = "#154b8b"

# Fallback number text for initial render; JS will overwrite with live values
auto_text = "Unknown" if autosomal_count is None else str(autosomal_count)

full_html_template = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Ancestor Register</title>
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { margin:0; padding:0; font-family: "Times New Roman", Georgia, serif; background:#ffffff; color:#222; font-size:14px; }
  a { color:%(LINK)s; text-decoration:none; } a:hover { text-decoration:underline; }

  .intro { padding:20px; text-align:center; }
  .intro h2 { margin:0 0 6px 0; font-size:24px; line-height:1.2; }
  .meta { font-size:12px; color:#555; margin:4px 0 10px 0; display:inline-block; }

  .toolbar { margin:8px auto 12px auto; display:flex; flex-wrap:wrap; gap:6px; justify-content:center; }
  .btn { display:inline-block; border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
         padding:4px 9px; border-radius:6px; font-size:13px; line-height:1.2; text-decoration:none;
         cursor:pointer; user-select:none; transition:background 0.2s, transform 0.1s; }
  .btn:hover { background:%(BTN_BG_H)s; transform:translateY(-1px); }
  .btn.light { background:#ffffff; color:#111; border-color:#bbb; }

  .output-table { max-height:75vh; overflow:auto; border:1px solid #ddd; margin:0 20px 24px 20px; }

  table.sortable { width:100%%; border-collapse:collapse; min-width:720px; table-layout:auto; }
  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; background:#ffffff; white-space:nowrap; }
  table.sortable th { position:sticky; top:0; z-index:2; text-align:left; cursor:pointer; background:%(TH_BG)s; box-shadow:0 1px 0 #ccc; }
  table.sortable tr#first-row td { border-top:2px solid #999 !important; }

  #searchBox { padding:4px 8px; font-size:13px; border:1px solid #bbb; border-radius:6px; outline:none; }

  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px;
                 border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
                 border-radius:6px; font-size:12px; display:none; z-index:9999; cursor:pointer; }
  .back-to-top:hover { background:%(BTN_BG_H)s; }

  @media screen and (max-width: 820px) {
    .intro { padding:14px; }
    .output-table { margin:0 12px 20px 12px; }
    .intro h2 { font-size:20px; }
    table.sortable { min-width:560px; }
  }
</style>
<script type="text/javascript">
//<![CDATA[
// Basic helpers
function _cellText(cell){
  var t = (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').replace(/^\\s+|\\s+$/g,'').toLowerCase();
  return t;
}
function _asNumber(s){
  var m = (s||'').replace(/[^0-9.\\-]/g,'');
  if(m.length===0) return NaN;
  var v = parseFloat(m);
  return isNaN(v) ? NaN : v;
}

// Sorting
function sortTableByColumn(tbl, colIndex, dirAsc){
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var tb = tbl.tBodies[0];
  var rows = Array.prototype.slice.call(tb.rows || []);
  rows.sort(function(a,b){
    var A = _cellText(a.cells[colIndex] || null);
    var B = _cellText(b.cells[colIndex] || null);
    var nA = _asNumber(A), nB = _asNumber(B);
    if(!isNaN(nA) && !isNaN(nB)){ return dirAsc ? (nA - nB) : (nB - nA); }
    if(A < B) return dirAsc ? -1 : 1;
    if(A > B) return dirAsc ?  1 : -1;
    return 0;
  });
  var frag = document.createDocumentFragment();
  for(var i=0;i<rows.length;i++){ frag.appendChild(rows[i]); }
  tb.appendChild(frag);
  updateShowingCount();
}
function bindHeaderSort(){
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
  var ths = tbl.tHead.rows[0].cells || [];
  for(var i=0;i<ths.length;i++){
    (function(idx){
      var th = ths[idx];
      var dirAsc = true;
      th.addEventListener('click', function(){
        for(var j=0;j<ths.length;j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[\\u2191\\u2193]$/, ''); }
        sortTableByColumn(tbl, idx, dirAsc);
        th.innerHTML = th.innerHTML.replace(/\\s+[\\u2191\\u2193]$/, '') + (dirAsc ? ' \\u2191' : ' \\u2193');
        dirAsc = !dirAsc;
      }, false);
    })(i);
  }
}

// Filter + live showing count
function filterTable(){
  var q = (document.getElementById('searchBox').value || '').toLowerCase();
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  for(var i=0;i<rows.length;i++){
    var cells = rows[i].cells, hit=false;
    for(var j=0;j<cells.length;j++){
      var txt = (cells[j].textContent || cells[j].innerText || '').toLowerCase();
      if(txt.indexOf(q) > -1){ hit=true; break; }
    }
    rows[i].style.display = hit ? '' : 'none';
  }
  updateShowingCount();
}
function updateShowingCount(){
  var el = document.getElementById('showing-count');
  var tbl = document.getElementById('refactor-table');
  if(!(el && tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  var vis = 0;
  for(var i=0;i<rows.length;i++){ if(rows[i].style.display !== 'none') vis++; }
  el.textContent = vis;
}

// Dynamic stamps
function z(n){ return (n<10 ? '0' : '') + n; }
function stampLastUpdated(){
  var el=document.getElementById('last-updated'); if(!el) return;
  var d=new Date(document.lastModified || new Date());
  el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());
}
function formatWithCommas(n){
  try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
}
function loadAutoCount(){
  var el=document.getElementById('auto-count'); if(!el) return;
  var url='{COUNT_URL}';
  try{
    var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
    xhr.onreadystatechange=function(){ if(xhr.readyState===4){
      if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; }
    }};
    xhr.send(null);
  }catch(e){ el.textContent='(unavailable)'; }
}

// Back-to-top
function bindBackToTop(){
  var btn = document.getElementById('back-to-top');
  if(!btn) return;
  function toggle(){ btn.style.display = (window.scrollY > 200) ? 'block' : 'none'; }
  toggle(); window.addEventListener('scroll', toggle, {passive:true});
  btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
}

document.addEventListener('DOMContentLoaded', function(){
  bindHeaderSort();
  bindBackToTop();
  stampLastUpdated();
  loadAutoCount();
  updateShowingCount();
}, false);
//]]>
</script>
</head>
<body>
  <div class="intro">
    <h2>Ancestor Register</h2>
    <div class="meta">
      <a href="%(HOME)s" target="_blank" rel="noopener">Home</a>
      &nbsp;|&nbsp; Last updated: <span id="last-updated">%(UPDATED)s</span>
      &nbsp;|&nbsp; Autosomal matches: <span id="auto-count">%(AUTO)s</span>
      &nbsp;|&nbsp; Showing: <span id="showing-count">0</span>
    </div>
    <div class="toolbar">
      <a class="btn" href="%(HOME)s" target="_blank" rel="noopener">&larr; Back to Main Register</a>
      <input type="text" id="searchBox" class="btn light" placeholder="Search this page&hellip;" oninput="filterTable()" />
    </div>
  </div>

  <div class="output-table" id="table-container">
    <!-- TABLE_PLACEHOLDER -->
  </div>

  <div class="back-to-top" id="back-to-top">&#9650; Top</div>
</body>
</html>""" % {
    "BTN_BG": _BTN_BG, "BTN_BG_H": _BTN_BG_H, "TH_BG": _TH_BG, "LINK": _LINK,
    "HOME": HOME_URL, "UPDATED": _html.escape(updated_fallback), "AUTO": _html.escape(auto_text)
}

# Build table HTML and mark first row
table_html = df.to_html(index=False, border=1, classes="sortable", table_id="refactor-table")
table_html = table_html.replace("<tbody>\n<tr>", "<tbody>\n<tr id=\"first-row\">", 1)

# Inject table and JS count URL
final_html = full_html_template.replace("<!-- TABLE_PLACEHOLDER -->", table_html)
final_html = final_html.replace("{COUNT_URL}", COUNT_PUBLIC_URL)
# ====== CUT STOP  [4/6] XHTML TEMPLATE + TABLE (Dynamic header + counts + search) ===============
#####
#####
#####
#####
#####


# ====== CUT START [5/6] SAVE LOCALLY + FTP UPLOAD ===============================================
# Save locally (iso-8859-15 safe)
try:
    with open(OUTPUT_NAME, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(final_html)
    print("[OK] Saved locally:", OUTPUT_NAME)
except Exception as e:
    print("[ERROR] Saving local file failed:", e)
    traceback.print_exc()

# Upload if credentials exist
ftp_host = os.environ.get('FTP_HOST')
ftp_user = os.environ.get('FTP_USER')
ftp_pass = os.environ.get('FTP_PASS')
ftp_port = os.environ.get('FTP_PORT', '21')
ftp_dir  = os.environ.get('FTP_DIR', '')

if all([ftp_host, ftp_user, ftp_pass]):
    print("[INFO] Attempting FTP upload...")
    try:
        socket.setdefaulttimeout(30)
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(ftp_host, int(ftp_port))
            ftps.login(ftp_user, ftp_pass)
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass
            # Navigate to target directory if provided
            if ftp_dir.strip('/'):
                for p in [p for p in ftp_dir.split('/') if p]:
                    try: ftps.cwd(p)
                    except Exception:
                        try: ftps.mkd(p)
                        except Exception: pass
                        ftps.cwd(p)
            # Replace remote file
            try: ftps.delete(OUTPUT_NAME)
            except Exception:
                pass
            with open(OUTPUT_NAME, "rb") as fh:
                ftps.storbinary(f"STOR {OUTPUT_NAME}", fh)
            print("[OK] Uploaded:", OUTPUT_NAME, "to", ftps.pwd())
    except Exception as e:
        print("[ERROR] FTP upload failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload: missing FTP credentials.")
# ====== CUT STOP  [5/6] SAVE LOCALLY + FTP UPLOAD ===============================================
#####
#####
#####
#####
#####


# ====== CUT START [6/6] PERSIST COUNT + DONE ====================================================
if autosomal_count is not None:
    try:
        with open("autosomal_count_prev.txt", "w") as f:
            f.write(str(autosomal_count))
        print("[OK] Persisted autosomal_count_prev.txt")
    except Exception as e:
        print("[WARN] Could not persist autosomal count:", e)

print("\n--- Script Finished ---")
# ====== CUT STOP  [6/6] PERSIST COUNT + DONE ====================================================
#####
#####
#####
#####
#####


[OK] Loaded CSV: final_combined_df_with_value_labels.csv rows=7, cols=9
Pulling resolver CSV from server...
Resolver saved: /content/match_to_unmasked.csv
[OK] Column B -> C mapping: 7 / 7 unmatched: 0
[OK] Saved locally: yates_ancestor_register.htm
[INFO] Attempting FTP upload...
[OK] Uploaded: yates_ancestor_register.htm to /
[OK] Persisted autosomal_count_prev.txt

--- Script Finished ---


In [ ]:
# EXP

import os
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo
from ftplib import FTP_TLS

# ── CONFIG ───────────────────────────────────────────────────────────────
info_csv   = "/content/haplogroup_info.csv"
user_csv   = "/content/y_dna_user_detail.csv"
output_csv = "/content/y_dna_grid.csv"
output_htm = "/content/y_dna_grid.htm"

# ── 1) Load & prepare haplogroup info ───────────────────────────────────
df_info = pd.read_csv(info_csv)
if "Date" in df_info.columns:
    df_info.rename(columns={"Date": "Era"}, inplace=True)
df_info = df_info.loc[df_info["Haplogroup"].drop_duplicates().index]
hap_order = df_info["Haplogroup"].tolist()
era_map   = dict(zip(df_info["Haplogroup"], df_info.get("Era", [""] * len(df_info))))

# ── 2) Load user detail table ───────────────────────────────────────────
df_users = pd.read_csv(user_csv)
if "User_ID" not in df_users.columns:
    df_users.rename(columns={df_users.columns[0]: "User_ID"}, inplace=True)
user_chains = [
    [str(v) for v in row.drop(labels=["User_ID"]).tolist() if pd.notna(v) and str(v).strip()]
    for _, row in df_users.iterrows()
]

# ── 3) Insert new SNPs after parent ──────────────────────────────────────
for chain in user_chains:
    prev = None
    for h in chain:
        if prev and h not in hap_order:
            idx = hap_order.index(prev)
            hap_order.insert(idx + 1, h)
        prev = h
# Build final eras list
eras = [era_map.get(h, "") for h in hap_order]

# ── 4) Build horizontal grid DataFrame ───────────────────────────────────
for h in hap_order:
    if h not in df_users.columns:
        df_users[h] = ""
df_grid_h = df_users[["User_ID"] + hap_order]

# ── 5) Transform to vertical layout ─────────────────────────────────────
df_vert = df_grid_h.set_index("User_ID").T
# Insert Era as first column
df_vert.insert(0, 'Era', eras)
df_vert.index.name = 'SNP'
df_grid = df_vert.reset_index()

# ── 6) Save vertical CSV ─────────────────────────────────────────────────
df_grid.to_csv(output_csv, index=False)
print(f"✅ Vertical grid CSV saved to {output_csv}")

# ── 7) Generate XHTML (vertical) ────────────────────────────────────────
now = datetime.now(ZoneInfo("America/New_York"))
ts  = now.strftime("%-m/%-d/%y, %-I:%M %p EDT")

template = '''<!DOCTYPE html>
<html><head><meta charset="UTF-8"><title>Yates Y-DNA Grid</title>
<style>
  body { background:#faf9d3; font-family:Arial,Helvetica,sans-serif; font-size:14px; }
  table { width:100%; border:1px solid #333; border-collapse:collapse; table-layout:auto; }
  th { background:#333; color:#fff; padding:6px; border:1px solid #999; }
  .era { background:#666; color:#eee; padding:6px; border:1px solid #999; font-size:0.9em; }
  td { padding:6px; border:1px solid #999; text-align:center; white-space:nowrap; }
  .match { background:#fff; }
  .blank { background:#ccc; color:#ccc; }
</style>
</head><body>
  <h1 style="text-align:center;">Yates Y-DNA Grid</h1>
  <table>
'''  # end template

# Build header row
cols = df_grid.columns.tolist()
header_html = '<tr><th>SNP</th><th>Era</th>' + ''.join(f'<th>{u}</th>' for u in cols[2:]) + '</tr>'

# Build data rows
rows_html = []
for _, row in df_grid.iterrows():
    cells = []
    for u in cols[2:]:
        v = row[u]
        if pd.isna(v) or not str(v).strip():
            cells.append('<td class="blank">–</td>')
        else:
            cells.append(f'<td class="match">{v}</td>')
    rows_html.append(f'<tr><td>{row["SNP"]}</td><td class="era">{row["Era"]}</td>' + ''.join(cells) + '</tr>')

# Combine and save HTML
html = template + header_html + '\n' + '\n'.join(rows_html) + f'''
  </table>
  <p style="text-align:right;font-size:0.9em;">Updated: {ts}</p>
</body>
</html>'''
with open(output_htm, 'w', encoding='utf-8') as f:
    f.write(html)
print(f"✅ Vertical XHTML Grid saved to {output_htm}")

# ── 8) FTP Upload ───────────────────────────────────────────────────────
ftp = FTP_TLS()
ftp.connect(os.environ['FTP_HOST'], int(os.environ.get('FTP_PORT',21)))
ftp.login(os.environ['FTP_USER'], os.environ['FTP_PASS'])
ftp.prot_p()
for path in [output_csv, output_htm]:
    name = os.path.basename(path)
    try: ftp.delete(name)
    except: pass
    with open(path,'rb') as fp:
        ftp.storbinary(f"STOR {name}", fp)
ftp.quit()
print("✅ Uploaded to server.")



✅ Vertical grid CSV saved to /content/y_dna_grid.csv
✅ Vertical XHTML Grid saved to /content/y_dna_grid.htm
✅ Uploaded to server.


<h2 id="cell-end">END - Runner</h2>
<a href="#ons-top">Back to top</a>


In [ ]:
# Y-DNA cell 1

# === Cell 1: New user settings ===
USER_ID       = 'I56217'  # the new column header
PATH_STRING   = (      # the SNP chain for this user
    "R-M207 > R-M173 > R-M343 > R-M269 > R-FT266064 > R-FT266579 > R-FTF17042"
)
INSERT_MISSING = True       # if True, adds any SNPs from PATH_STRING that aren't yet rows
MASTER_CSV     = '/content/y_dna_user_detail_combo.csv'
UPDATED_CSV    = '/content/y_dna_user_detail_combo_updated.csv'


In [ ]:
# Cell 2: Load → Append User → Save

import pandas as pd

# 1) Load the existing master CSV
df = pd.read_csv(MASTER_CSV)

# 2) Normalize the first column name to 'SNP' for easy matching
first_col = df.columns[0]
if first_col != 'SNP':
    df.rename(columns={first_col: 'SNP'}, inplace=True)

# 3) Parse the new user's SNP chain
chain = PATH_STRING.split('>')

# 4) Optionally insert any SNPs not yet present (appends at bottom)
if INSERT_MISSING:
    missing = [s for s in chain if s not in df['SNP'].values]
    if missing:
        df = pd.concat([df, pd.DataFrame([{'SNP': s} for s in missing])],
                       ignore_index=True)

# 5) Create the new user column in the next free position
df[USER_ID] = ''

# 6) Populate: copy the SNP value into that column where it matches the chain
df.loc[df['SNP'].isin(chain), USER_ID] = df['SNP']

# 7) Save the updated CSV back to /content
df.to_csv(UPDATED_CSV, index=False)
print(f"✅ Updated CSV saved to {UPDATED_CSV}")


✅ Updated CSV saved to /content/y_dna_user_detail_combo_updated.csv
